# Regression of Vote Characteristics on CAR

In [40]:
import stata_setup

stata_setup.config('/Applications/StataNow', 'mp')

In [41]:
%%stata

clear all
set more off
set varabbrev off
version 18

* Paths
global PROCESSED_DATA "processed_data"
global TABLES "tables"


. 
. clear all

. set more off

. set varabbrev off

. version 18

. 
. * Paths
. global PROCESSED_DATA "processed_data"

. global TABLES "tables"

. 


In [ ]:
%%stata
foreach stage in created end {
    import delimited using "$PROCESSED_DATA/event_study_panel_`stage'.csv", clear

    * Parse date
    gen day = date(date, "YMD")
    format day %td

    * Create year variable
    gen year = year(day)

    * Create quarter variable
    gen quarter = qofd(day)
    format quarter %tq

    * Ensure index is numeric (safeguard if it came in as string)
    capture confirm numeric variable index
    if _rc {
        destring index, replace ignore("[]")
    }

    * Encode categorical variables
    encode gecko_id, gen(token)
    encode space,    gen(dao)

    egen token_year = group(token year)
    egen token_quarter = group(token quarter)

    * Define windows and specs
    local vars    non_whale_victory_vn non_whale_victory_vp non_whale_victory_vp_vn

    * Label variables
    label var non_whale_victory_vn "\${\it Small Win}^{vn}_{i,t}\$"
    label var non_whale_victory_vp "\${\it Small Win}^{vp}_{i,t}\$"
    label var non_whale_victory_vp_vn "\${\it Small Win}^{vp,vn}_{i,t}\$"

    eststo clear

    keep if index == `window'

    foreach v of local vars {

        * Run regression
        reghdfe car `v', absorb(year dao) vce(cluster token)
        eststo car_`v'

        * Tag FE rows (Y/N) for this spec
        estadd local fe_token "Y"
        estadd local fe_time  "Y"
    }

    * Export a single big LaTeX table
    esttab                                                         ///
        car_non_whale_victory_vn car_non_whale_victory_vp car_non_whale_victory_vp_vn ///
        using "$TABLES/reg_car_win_`stage'.tex", replace         ///
        se star(* 0.10 ** 0.05 *** 0.01) label nogaps nocon        ///
        nonotes booktabs nomtitles                                 ///
        b(%9.3f) se(%9.2f)                                         ///
        mgroups("CAR[-5,5]",                           ///
                span                                               ///
                pattern(1 0 0 0)                           ///
                prefix(\multicolumn{@span}{c}{)                    ///
                suffix(})                                          ///
                erepeat(\cmidrule(lr){@span}) )                    ///
        substitute("\_" "_")                                       ///
        stats(fe_token fe_time N r2_a,                             ///      
              fmt(0 0 %9.0fc %9.3f) labels("Token FE" "Year FE" "Observations" "Adjusted R²"))
}


. foreach stage in created end {
  2.     import delimited using "$PROCESSED_DATA/event_study_panel_`stage'.csv"
> , clear
  3. 
.     * Parse date
.     gen day = date(date, "YMD")
  4.     format day %td
  5. 
.     gen year = year(day)
  6.     encode gecko_id, gen(token)
  7.     encode space,    gen(dao)
  8. 
.     * Define dependent variables
.     local wins non_whale_victory_vn non_whale_victory_vp non_whale_victory_vp
> _vn
  9. 
.     * Define independent variables
.     local indep n_choices duration quorum
 10. 
.     eststo clear
 11. 
.     foreach y of local wins {
 12.         * Logistic regression (clustered by DAO)
.         logit `y' `indep' i.year, vce(cluster dao)
 13.         eststo `y'
 14. 
.         * Add indicators for FE presence
.         estadd local fe_year "Y"
 15.         estadd local fe_dao "Clustered"
 16.     }
 17. 
.     * Export LaTeX table
.     esttab                                                     ///
>         non_whale_victory_vn        